In [1]:
import numpy as np
import mediapipe as mp
import cv2
import math

from tensorflow.keras.models import load_model
from IPython.display import clear_output


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# body_positions CONTIENE LAS POSICIONES DE LOS LANDMARKS, SEGUIDO DE "TRUE" SI SE HA GENERADO POR PREDICCIÓN DEL NUEVO MODELO, O "FALSE" SI SE HA GENERADO POR EL MODELO MEDIAPIPE

body_positions = {"left wrist" :((0,0),False), "left elbow" :((0,0),False), 
                  "right wrist":((0,0),False), "right elbow":((0,0),False),
                  "left ankle" :((0,0),False), "left knee"  :((0,0),False),
                  "right ankle":((0,0),False), "right knee" :((0,0),False),}

In [3]:
def angle3coord(a=np.array([0,0]), b=np.array([0,0])):   # ANGULOS EN RADIANES
    c=np.array([b[0]+1,b[1]])
    ang = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
    return (ang + 2*math.pi) if ang < 0 else ang

In [4]:
left_arm_net_sin      , left_arm_net_cos       = load_model('models/left_arm_net_sin.h5')      , load_model('models/left_arm_net_cos.h5')
left_calf_net_sin     , left_calf_net_cos      = load_model('models/left_calf_net_sin.h5')     , load_model('models/left_calf_net_cos.h5')
left_forearm_net_sin  , left_forearm_net_cos   = load_model('models/left_forearm_net_sin.h5')  , load_model('models/left_forearm_net_cos.h5')
left_leg_net_sin      , left_leg_net_cos       = load_model('models/left_leg_net_sin.h5')      , load_model('models/left_leg_net_cos.h5')
left_shoulder_net_sin , left_shoulder_net_cos  = load_model('models/left_shoulder_net_sin.h5') , load_model('models/left_shoulder_net_cos.h5')
right_arm_net_sin     , right_arm_net_cos      = load_model('models/right_arm_net_sin.h5')     , load_model('models/right_arm_net_cos.h5')
right_calf_net_sin    , right_calf_net_cos     = load_model('models/right_calf_net_sin.h5')    , load_model('models/right_calf_net_cos.h5')
right_forearm_net_sin , right_forearm_net_cos  = load_model('models/right_forearm_net_sin.h5') , load_model('models/right_forearm_net_cos.h5')
right_leg_net_sin     , right_leg_net_cos      = load_model('models/right_leg_net_sin.h5')     , load_model('models/right_leg_net_cos.h5')
right_shoulder_net_sin, right_shoulder_net_cos = load_model('models/right_shoulder_net_sin.h5'), load_model('models/right_shoulder_net_cos.h5')

In [5]:
left_forearm_dist, left_arm_dist, right_arm_dist, right_forearm_dist, left_hand_dist, right_hand_dist = 0, 0, 0, 0, 0, 0
left_leg_dist, left_calf_dist, right_leg_dist, right_calf_dist, left_foot_dist, right_foot_dist       = 0, 0, 0, 0, 0, 0

def edistance(a,b):
    return np.linalg.norm(np.array([keypoints[a].x * width, keypoints[a].y * height])-np.array([keypoints[b].x * width, keypoints[b].y * height]))
    
def calculateposition(a,xx,yy,dist):
    return (int((keypoints[a].x * width) + (xx * dist)),int((keypoints[a].y * height) - (yy * dist)))

In [13]:
cap = cv2.VideoCapture("Untitled.mp4")
conf = 0.82
confizq= 0.78
confdist = 0.95
wind1 = 'Body occlusion'
wind2 = 'Skeleton'

frame_array_cos = np.zeros((15,29), dtype='float64')
frame_array_sin = np.zeros((15,29), dtype='float64')
current_frame_sin, current_frame_cos = np.zeros((1,29), dtype='float64'), np.zeros((1,29), dtype='float64')
bg_img = np.zeros((512, 512, 3), dtype = "uint8")

c_exposed = (0,255,0)
c_covered = (0,0,255)

border = 150

with mp_pose.Pose(
    min_detection_confidence=conf,
    min_tracking_confidence=conf) as pose:
  while cap.isOpened():

    background = np.zeros((580,740,3), dtype='float64')

    if cv2.waitKey(5) & 0xFF == 27:
      cap.release()  
      cv2.destroyAllWindows()
      break

    success, image = cap.read()
    if not success:
      print("Ignoring video input...")
      # If loading a video, use 'break' instead of 'continue'.
      break

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB. 
    
    image = np.pad(image,((border, border), (border, border), (0, 0)),mode='constant', constant_values=0)
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    
    results = pose.process(image)

    # Draw the pose annotation on the image.
    # image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    height, width = image.shape[:2]
    
    
    
    # CONVERSIÓN DE LANNMARKS A NUMPY ARRAY
    try:
      keypoints = results.pose_landmarks.landmark
      clear_output(wait=True)


      # VENTANA CAMARA


        # LLENADO Y ACTUALIZACION DE FRAME_ARRAY. CORRESPONDEN A LAS FUNCIONES SIN() Y COS() DE LOS ANGULOS DE LOS ULTIMOS 15 FRAMES

      
      frame_array_cos = np.delete(frame_array_cos, 0, axis=0)
      frame_array_cos = np.append(frame_array_cos,current_frame_cos, axis=0)

      frame_array_sin = np.delete(frame_array_sin, 0, axis=0)
      frame_array_sin = np.append(frame_array_sin,current_frame_sin, axis=0)


      current_frame_sin = np.array([[math.sin(angle3coord((keypoints[14].x, keypoints[14].y), (keypoints[12].x, keypoints[12].y))),math.sin(angle3coord((keypoints[16].x, keypoints[16].y), (keypoints[14].x, keypoints[14].y))),math.sin(angle3coord((keypoints[22].x, keypoints[22].y), (keypoints[16].x, keypoints[16].y))),
                                math.sin(angle3coord((keypoints[20].x, keypoints[20].y), (keypoints[16].x, keypoints[16].y))),math.sin(angle3coord((keypoints[18].x, keypoints[18].y), (keypoints[16].x, keypoints[16].y))),math.sin(angle3coord((keypoints[13].x, keypoints[13].y), (keypoints[11].x, keypoints[11].y))),
                                math.sin(angle3coord((keypoints[15].x, keypoints[15].y), (keypoints[13].x, keypoints[13].y))),math.sin(angle3coord((keypoints[21].x, keypoints[21].y), (keypoints[15].x, keypoints[15].y))),math.sin(angle3coord((keypoints[19].x, keypoints[19].y), (keypoints[15].x, keypoints[15].y))),
                                math.sin(angle3coord((keypoints[17].x, keypoints[17].y), (keypoints[15].x, keypoints[15].y))),math.sin(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[11].x, keypoints[11].y))),math.sin(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[24].x, keypoints[24].y))),
                                math.sin(angle3coord((keypoints[11].x, keypoints[11].y), (keypoints[23].x, keypoints[23].y))),math.sin(angle3coord((keypoints[24].x, keypoints[24].y), (keypoints[23].x, keypoints[23].y))),math.sin(angle3coord((keypoints[26].x, keypoints[26].y), (keypoints[24].x, keypoints[24].y))),
                                math.sin(angle3coord((keypoints[25].x, keypoints[25].y), (keypoints[23].x, keypoints[23].y))),math.sin(angle3coord((keypoints[28].x, keypoints[28].y), (keypoints[26].x, keypoints[26].y))),math.sin(angle3coord((keypoints[27].x, keypoints[27].y), (keypoints[25].x, keypoints[25].y))),
                                math.sin(angle3coord((keypoints[32].x, keypoints[32].y), (keypoints[28].x, keypoints[28].y))),math.sin(angle3coord((keypoints[31].x, keypoints[31].y), (keypoints[27].x, keypoints[27].y))),math.sin(angle3coord((keypoints[32].x, keypoints[32].y), (keypoints[30].x, keypoints[30].y))),
                                math.sin(angle3coord((keypoints[30].x, keypoints[30].y), (keypoints[28].x, keypoints[28].y))),math.sin(angle3coord((keypoints[31].x, keypoints[31].y), (keypoints[29].x, keypoints[29].y))),math.sin(angle3coord((keypoints[29].x, keypoints[29].y), (keypoints[27].x, keypoints[27].y))),
                                math.sin(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[10].x, keypoints[10].y))),math.sin(angle3coord((keypoints[11].x, keypoints[11].y), (keypoints[9].x, keypoints[9].y)))  ,math.sin(angle3coord((keypoints[10].x, keypoints[10].y), (keypoints[9].x, keypoints[9].y))),
                                math.sin(angle3coord((keypoints[6].x, keypoints[6].y)  , (keypoints[4].x, keypoints[4].y)))  ,math.sin(angle3coord((keypoints[1].x, keypoints[1].y), (keypoints[3].x, keypoints[3].y)))
                                ]])

      current_frame_cos = np.array([[math.cos(angle3coord((keypoints[14].x, keypoints[14].y), (keypoints[12].x, keypoints[12].y))),math.cos(angle3coord((keypoints[16].x, keypoints[16].y), (keypoints[14].x, keypoints[14].y))),math.cos(angle3coord((keypoints[22].x, keypoints[22].y), (keypoints[16].x, keypoints[16].y))),
                                math.cos(angle3coord((keypoints[20].x, keypoints[20].y), (keypoints[16].x, keypoints[16].y))),math.cos(angle3coord((keypoints[18].x, keypoints[18].y), (keypoints[16].x, keypoints[16].y))),math.cos(angle3coord((keypoints[13].x, keypoints[13].y), (keypoints[11].x, keypoints[11].y))),
                                math.cos(angle3coord((keypoints[15].x, keypoints[15].y), (keypoints[13].x, keypoints[13].y))),math.cos(angle3coord((keypoints[21].x, keypoints[21].y), (keypoints[15].x, keypoints[15].y))),math.cos(angle3coord((keypoints[19].x, keypoints[19].y), (keypoints[15].x, keypoints[15].y))),
                                math.cos(angle3coord((keypoints[17].x, keypoints[17].y), (keypoints[15].x, keypoints[15].y))),math.cos(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[11].x, keypoints[11].y))),math.cos(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[24].x, keypoints[24].y))),
                                math.cos(angle3coord((keypoints[11].x, keypoints[11].y), (keypoints[23].x, keypoints[23].y))),math.cos(angle3coord((keypoints[24].x, keypoints[24].y), (keypoints[23].x, keypoints[23].y))),math.cos(angle3coord((keypoints[26].x, keypoints[26].y), (keypoints[24].x, keypoints[24].y))),
                                math.cos(angle3coord((keypoints[25].x, keypoints[25].y), (keypoints[23].x, keypoints[23].y))),math.cos(angle3coord((keypoints[28].x, keypoints[28].y), (keypoints[26].x, keypoints[26].y))),math.cos(angle3coord((keypoints[27].x, keypoints[27].y), (keypoints[25].x, keypoints[25].y))),
                                math.cos(angle3coord((keypoints[32].x, keypoints[32].y), (keypoints[28].x, keypoints[28].y))),math.cos(angle3coord((keypoints[31].x, keypoints[31].y), (keypoints[27].x, keypoints[27].y))),math.cos(angle3coord((keypoints[32].x, keypoints[32].y), (keypoints[30].x, keypoints[30].y))),
                                math.cos(angle3coord((keypoints[30].x, keypoints[30].y), (keypoints[28].x, keypoints[28].y))),math.cos(angle3coord((keypoints[31].x, keypoints[31].y), (keypoints[29].x, keypoints[29].y))),math.cos(angle3coord((keypoints[29].x, keypoints[29].y), (keypoints[27].x, keypoints[27].y))),
                                math.cos(angle3coord((keypoints[12].x, keypoints[12].y), (keypoints[10].x, keypoints[10].y))),math.cos(angle3coord((keypoints[11].x, keypoints[11].y), (keypoints[9].x, keypoints[9].y)))  ,math.cos(angle3coord((keypoints[10].x, keypoints[10].y), (keypoints[9].x, keypoints[9].y))),
                                math.cos(angle3coord((keypoints[6].x, keypoints[6].y)  , (keypoints[4].x, keypoints[4].y)))  ,math.cos(angle3coord((keypoints[1].x, keypoints[1].y), (keypoints[3].x, keypoints[3].y)))
                                ]])
      


      # VISIBILIDAD DE PARTES DEL CUERPO // MAPEO DE ESQUELETO
      
      
      if keypoints[12].visibility < confizq or keypoints[14].visibility < confizq:    # BRAZO IZQUIERDO
        cv2.line(bg_img,(185,156),(149,195),(0,0,255),8)
        x1 = left_arm_net_cos.predict(np.array([frame_array_cos.compress([1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y1 = left_arm_net_sin.predict(np.array([frame_array_sin.compress([1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,0] = x1
        current_frame_sin[0,0] = y1
        frame_array_cos[-1,0] = x1
        frame_array_sin[-1,0] = y1
        landmark_left_arm = calculateposition(12, x1, y1, left_arm_dist)
        cv2.line(image,(int(keypoints[12].x * width),int(keypoints[12].y * height)),landmark_left_arm,c_covered,3)

      else:
        cv2.line(bg_img,(185,156),(149,195),(0,255,0),8)
        landmark_left_arm = (int(keypoints[14].x * width),int(keypoints[14].y * height))
        cv2.line(image,(int(keypoints[12].x * width),int(keypoints[12].y * height)),landmark_left_arm,c_exposed,3)
        if(keypoints[12].visibility >= confdist and keypoints[14].visibility >= confdist):
          left_arm_dist = edistance(12,14)
      
      body_positions['left elbow'] = (landmark_left_arm,keypoints[12].visibility < confizq or keypoints[14].visibility < confizq)






      if keypoints[14].visibility < conf or keypoints[16].visibility < conf:    # ANTEBRAZO IZQUIERDO
        cv2.line(bg_img,(110,177),(149,195),(0,0,255),8)
        x2 = left_forearm_net_cos.predict(np.array([frame_array_cos.compress([1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y2 = left_forearm_net_sin.predict(np.array([frame_array_sin.compress([1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,1] = x2
        current_frame_sin[0,1] = y2
        frame_array_cos[-1,1] = x2
        frame_array_sin[-1,1] = y2
        landmark_left_forearm = calculateposition(14, x2, y2, left_forearm_dist)
        cv2.line(image,landmark_left_arm,landmark_left_forearm,c_covered,3)
      
      else:
        cv2.line(bg_img,(110,177),(149,195),(0,255,0),8)
        landmark_left_forearm = (int(keypoints[16].x * width),int(keypoints[16].y * height))
        cv2.line(image,landmark_left_arm, landmark_left_forearm, c_exposed,3)
        if(keypoints[14].visibility >= confdist and keypoints[16].visibility >= confdist):
          left_forearm_dist = edistance(16,14)
        
      body_positions['left wrist'] = (landmark_left_forearm, keypoints[14].visibility < conf or keypoints[16].visibility < conf)





      
      if keypoints[16].visibility < conf or keypoints[20].visibility < conf:  # MANO IZQUIERDA
        cv2.circle(bg_img, (92,165), 2, (0,0,255), 8)
        landmark_left_hand = (int(landmark_left_forearm[0] + (current_frame_cos[0,1]*left_hand_dist)),int(landmark_left_forearm[1] - (current_frame_sin[0,1]*left_hand_dist)))
        cv2.line(image,landmark_left_forearm, landmark_left_hand, c_covered,5)
      
      else:
        cv2.circle(bg_img, (92,165), 2, (0,255,0), 8)
        cv2.line(image,(int(keypoints[16].x * width),int(keypoints[16].y * height)),(int(keypoints[20].x * width),int(keypoints[20].y * height)),c_exposed,6)
        if(keypoints[16].visibility >= confdist and keypoints[20].visibility >= confdist):
          left_hand_dist = edistance(16,20)

        

      
      


      if keypoints[11].visibility < conf or keypoints[13].visibility < conf:    # BRAZO DERECHO
        cv2.line(bg_img,(303,156),(339,195),(0,0,255),8)
        x3 = right_arm_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y3 = right_arm_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,5] = x3
        current_frame_sin[0,5] = y3
        frame_array_cos[-1,5] = x3
        frame_array_sin[-1,5] = y3
        landmark_right_arm = calculateposition(11, x3, y3, right_arm_dist)
        cv2.line(image,(int(keypoints[11].x * width),int(keypoints[11].y * height)),landmark_right_arm,c_covered,3)
      
      else:
        cv2.line(bg_img,(303,156),(339,195),(0,255,0),8)
        landmark_right_arm = (int(keypoints[13].x * width),int(keypoints[13].y * height))
        cv2.line(image,(int(keypoints[11].x * width),int(keypoints[11].y * height)),landmark_right_arm,c_exposed,3)
        if(keypoints[11].visibility >= confdist and keypoints[13].visibility >= confdist):
          left_arm_dist = edistance(11,13)

      body_positions['right elbow'] = (landmark_right_arm, keypoints[11].visibility < conf or keypoints[13].visibility < conf)






      if keypoints[13].visibility < conf or keypoints[15].visibility < conf:    # ANTEBRAZO DERECHO
        cv2.line(bg_img,(378,177),(339,195),(0,0,255),8)
        x4 = right_forearm_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y4 = right_forearm_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,6] = x4
        current_frame_sin[0,6] = y4
        frame_array_cos[-1,6] = x4
        frame_array_sin[-1,6] = y4
        landmark_right_forearm = calculateposition(13, x4, y4, right_forearm_dist)
        cv2.line(image,landmark_right_arm, landmark_right_forearm, c_covered,3)
      else:
        cv2.line(bg_img,(378,177),(339,195),(0,255,0),8)
        landmark_right_forearm = (int(keypoints[15].x * width),int(keypoints[15].y * height))
        cv2.line(image,landmark_right_forearm,landmark_right_arm,c_exposed,3)
        if(keypoints[13].visibility >= confdist and keypoints[15].visibility >= confdist):
          right_forearm_dist = edistance(13,15)

      body_positions['right wrist'] = (landmark_right_forearm, keypoints[13].visibility < conf or keypoints[15].visibility < conf)





      if keypoints[15].visibility < conf or keypoints[19].visibility < conf:  # MANO DERECHA
        cv2.circle(bg_img, (396,165), 2, (0,0,255), 8)
        cv2.line(image,landmark_right_forearm,
                       (int(landmark_right_forearm[0] + (current_frame_cos[0,6]*right_hand_dist)),int(landmark_right_forearm[1] - (current_frame_sin[0,6]*right_hand_dist))),c_covered,5)
      else:
        cv2.circle(bg_img, (396,165), 2, (0,255,0), 8)
        cv2.line(image,(int(keypoints[19].x * width),int(keypoints[19].y * height)),(int(keypoints[15].x * width),int(keypoints[15].y * height)),c_exposed,3)
        if(keypoints[15].visibility >= confdist and keypoints[19].visibility >= confdist):
          right_hand_dist = edistance(19,15)

     



      if keypoints[24].visibility < conf or keypoints[26].visibility < conf:    # PIERNA IZQUIERDA
        cv2.line(bg_img,(205,262),(186,334),(0,0,255),8)
        x5 = left_leg_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y5 = left_leg_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,14] = x5
        current_frame_sin[0,14] = y5
        frame_array_cos[-1,14] = x5
        frame_array_sin[-1,14] = y5
        landmark_left_leg = calculateposition(24, x5, y5, left_leg_dist)
        cv2.line(image,(int(keypoints[24].x * width),int(keypoints[24].y * height)),landmark_left_leg,c_covered,3)
      else:
        cv2.line(bg_img,(205,262),(186,334),(0,255,0),8)
        landmark_left_leg = (int(keypoints[26].x * width),int(keypoints[26].y * height))
        cv2.line(image,(int(keypoints[24].x * width),int(keypoints[24].y * height)),landmark_left_leg,c_exposed,3)
        if(keypoints[24].visibility >= confdist and keypoints[26].visibility >= confdist):
          left_leg_dist = edistance(24,26)

      body_positions['left knee'] = (landmark_left_leg, keypoints[24].visibility < conf or keypoints[26].visibility < conf)






      if keypoints[26].visibility < conf or keypoints[28].visibility < conf:    # PANTORRILLA IZQUIERDA
        cv2.line(bg_img,(205,413),(186,334),(0,0,255),8)
        x6 = left_calf_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1], axis=1)]))[0,0]
        y6 = left_calf_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,16] = x6
        current_frame_sin[0,16] = y6
        frame_array_cos[-1,16] = x6
        frame_array_sin[-1,16] = y6
        landmark_left_calf = calculateposition(26, x6, y6, left_calf_dist)
        cv2.line(image,landmark_left_calf, landmark_left_leg, c_covered,3)
      else:
        cv2.line(bg_img,(205,413),(186,334),(0,255,0),8)
        landmark_left_calf = (int(keypoints[28].x * width),int(keypoints[28].y * height))
        cv2.line(image,landmark_left_leg,landmark_left_calf,c_exposed,3)
        if(keypoints[26].visibility >= confdist and keypoints[28].visibility >= confdist):
          left_calf_dist = edistance(26,28)

      body_positions['left ankle'] = (landmark_left_calf, keypoints[26].visibility < conf or keypoints[28].visibility < conf)







      if keypoints[28].visibility < conf or keypoints[30].visibility < conf or keypoints[32].visibility < conf:   # PIE IZQUIERDO
        cv2.line(bg_img,(205,413),(173,413),(0,0,255),8)
      else:
        cv2.line(bg_img,(205,413),(173,413),(0,255,0),8)
        cv2.line(image,(int(keypoints[32].x * width),int(keypoints[32].y * height)),(int(keypoints[28].x * width),int(keypoints[28].y * height)),c_exposed,3)

    



      if keypoints[23].visibility < conf or keypoints[25].visibility < conf:    # PIERNA DERECHA
        cv2.line(bg_img,(283,262),(302,334),(0,0,255),8)
        x7 = right_leg_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,1,1,1,1,1], axis=1)]))[0,0]
        y7 = right_leg_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,15] = x7
        current_frame_sin[0,15] = y7
        frame_array_cos[-1,15] = x7
        frame_array_sin[-1,15] = y7
        landmark_right_leg = calculateposition(23, x7, y7, left_leg_dist)
        cv2.line(image,(int(keypoints[23].x * width),int(keypoints[23].y * height)),landmark_right_leg,c_covered,3)
      else:
        cv2.line(bg_img,(283,262),(302,334),(0,255,0),8)
        landmark_right_leg = (int(keypoints[25].x * width),int(keypoints[25].y * height))
        cv2.line(image,(int(keypoints[23].x * width),int(keypoints[23].y * height)),landmark_right_leg,c_exposed,3)
        if(keypoints[23].visibility >= confdist and keypoints[25].visibility >= confdist):
          right_leg_dist = edistance(23,25)

      body_positions['right knee'] = (landmark_right_leg, keypoints[23].visibility < conf or keypoints[25].visibility < conf)



      if keypoints[25].visibility < conf or keypoints[27].visibility < conf:    # PANTORRILLA DERECHA
        cv2.line(bg_img,(283,413),(302,334),(0,0,255),8)
        x8 = right_calf_net_cos.predict(np.array([frame_array_cos.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,1], axis=1)]))[0,0]
        y8 = right_calf_net_sin.predict(np.array([frame_array_sin.compress([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,1], axis=1)]))[0,0]
        current_frame_cos[0,17] = x8
        current_frame_sin[0,17] = y8
        frame_array_cos[-1,17] = x8
        frame_array_sin[-1,17] = y8
        landmark_right_calf = calculateposition(25, x8, y8, right_calf_dist)
        cv2.line(image,landmark_right_calf, landmark_right_leg, c_covered,3)
      else:
        cv2.line(bg_img,(283,413),(302,334),(0,255,0),8)
        landmark_right_calf = (int(keypoints[27].x * width),int(keypoints[27].y * height))
        cv2.line(image,landmark_right_leg,landmark_right_calf,c_exposed,3)
        if(keypoints[25].visibility >= confdist and keypoints[27].visibility >= confdist):
          right_calf_dist = edistance(25,27)

      body_positions['right ankle'] = (landmark_right_calf, keypoints[25].visibility < conf or keypoints[27].visibility < conf)



      if keypoints[27].visibility < conf or keypoints[29].visibility < conf or keypoints[31].visibility < conf:   # PIE DERECHO
        cv2.line(bg_img,(283,413),(315,413),(0,0,255),8)
      else:
        cv2.line(bg_img,(283,413),(315,413),(0,255,0),8)
        cv2.line(image,(int(keypoints[27].x * width),int(keypoints[27].y * height)),(int(keypoints[31].x * width),int(keypoints[31].y * height)),c_exposed,3)





      if keypoints[10].visibility < conf or keypoints[9].visibility < conf or keypoints[0].visibility < conf or keypoints[2].visibility < conf or keypoints[5].visibility < conf:     # CABEZA
        cv2.circle(bg_img, (244,95), 30, (0,0,255), 8)
        
      else:
        cv2.circle(bg_img, (244,95), 30, (0,255,0), 8)
        distance = edistance(1,9)
        cv2.circle(image, (int(keypoints[0].x * width),int(keypoints[0].y * height)), int(distance), c_exposed, 3)
        
      
      cv2.line(bg_img,(200,156),(288,156),(255,255,255),8)
      cv2.line(bg_img,(268,262),(220,262),(255,255,255),8)
      cv2.line(bg_img,(200,156),(220,262),(255,255,255),8)
      cv2.line(bg_img,(268,262),(288,156),(255,255,255),8)

      cv2.circle(bg_img, (53,409), 10, (0,0,0), 8)


    except AttributeError:
      keypoints = []
      cv2.circle(bg_img, (244,95), 30, (0,0,255), 8)
      cv2.line(bg_img,(283,413),(315,413),(0,0,255),8)
      cv2.line(bg_img,(283,413),(302,334),(0,0,255),8)
      cv2.line(bg_img,(185,156),(149,195),(0,0,255),8)
      cv2.line(bg_img,(283,262),(302,334),(0,0,255),8)
      cv2.line(bg_img,(205,413),(173,413),(0,0,255),8)
      cv2.line(bg_img,(110,177),(149,195),(0,0,255),8)
      cv2.line(bg_img,(205,413),(186,334),(0,0,255),8)
      cv2.line(bg_img,(205,262),(186,334),(0,0,255),8)
      cv2.circle(bg_img, (396,165), 2, (0,0,255), 8)
      cv2.circle(bg_img, (92,165), 2, (0,0,255), 8)
      cv2.line(bg_img,(303,156),(339,195),(0,0,255),8)
      cv2.line(bg_img,(378,177),(339,195),(0,0,255),8)
      cv2.line(bg_img,(200,156),(288,156),(255,255,255),8)
      cv2.line(bg_img,(268,262),(220,262),(255,255,255),8)
      cv2.line(bg_img,(200,156),(220,262),(255,255,255),8)
      cv2.line(bg_img,(268,262),(288,156),(255,255,255),8)
      cv2.circle(bg_img, (53,409), 10, (0,0,255), 8)
    
    
    cv2.imshow(wind1, image)
    cv2.imshow(wind2, bg_img)
    print(body_positions)
    
  
    

    

{'left wrist': ((526, 285), False), 'left elbow': ((510, 325), False), 'right wrist': ((615, 299), False), 'right elbow': ((603, 315), False), 'left ankle': ((522, 578), False), 'left knee': ((543, 488), False), 'right ankle': ((579, 574), False), 'right knee': ((595, 488), False)}


In [7]:
cap.release()
cv2.destroyAllWindows()